<a href="https://colab.research.google.com/github/YuanchengJin/CP1404/blob/master/Video_Image_Restoration_CodeFormer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img src="https://user-images.githubusercontent.com/14334509/179359809-bd45566a-486d-418f-83fa-67bbbba8c45c.png" height=120>
</p>

# CodeFormer Inference
## Towards Robust Blind Face Restoration with Codebook Lookup Transformer (NeurIPS 2022)
Shangchen Zhou, Kelvin C.K. Chan, Chongyi Li, Chen Change Loy

CodeFormer is a Transformer-based network for robust blind face restoration. It uses a learned discrete codebook to reduce uncertainty in mapping degraded inputs to desired outputs and generate high-quality faces. CodeFormer outperforms existing methods in quality and fidelity, adapting well to different degradation levels.

| Code Credits | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/github/stars/sczhou/CodeFormer?style=social)](https://github.com/sczhou/CodeFormer) |
| Original Colab | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1m52PNveE4PBhYrecj34cnpEeiHcC5LTb?usp=sharing) |
| 🚀 Online inference | [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/sczhou/CodeFormer) |
| 🚀 Online inference | [![Replicate](https://replicate.com/google-research/frame-interpolation/badge)](https://replicate.com/sczhou/codeformer)
 |
| 🔥 Discover More Colab Notebooks | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/InsightSolver-Colab/) |


# 1. Preparations
Before start, make sure that you choose
* Hardware Accelerator = GPU (in the Runtime menu -> Change runtime type)

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [8]:
# @title Setup
# Clone CodeFormer and enter the CodeFormer folder
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd CodeFormer

# # Set up the environment
# Install python dependencies
!pip install -q -r requirements.txt
!pip -q install ffmpeg-python
# Install basicsr
!python basicsr/setup.py develop

# Download the pre-trained model
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

# Visualization function
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 614 (delta 202), reused 176 (delta 176), pack-reused 322 (from 3)
Receiving objects: 100% (614/614), 17.31 MiB | 36.78 MiB/s, done.
Resolving deltas: 100% (296/296), done.
/content/CodeFormer
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/di

&nbsp;
***
&nbsp;
# 2. Test on your images or video 😀


In [9]:
# !find /content -name "ffmpeg.py"

# !pip uninstall -y ffmpeg
# !pip install ffmpeg-python

# import ffmpeg
# print(ffmpeg.__file__)


In [10]:
#@title Upload your own images or video
import os
from google.colab import files
import shutil

upload_folder = 'inputs/user_upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
    os.mkdir(upload_folder)

uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [11]:
# Alternatively, you can submit a video from Google Drive to this path and uncomment the line.
filename = "/content/CodeFormer/inputs/user_upload/video.mp4"

In [12]:
# !pip uninstall ffmpeg
# !pip install ffmpeg-python
import ffmpeg
print(dir(ffmpeg))

['Error', 'Stream', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_ffmpeg', '_filters', '_probe', '_run', '_utils', '_view', 'colorchannelmixer', 'compile', 'concat', 'crop', 'dag', 'drawbox', 'drawtext', 'filter', 'filter_', 'filter_multi_output', 'get_args', 'hflip', 'hue', 'input', 'merge_outputs', 'nodes', 'output', 'overlay', 'overwrite_output', 'probe', 'run', 'run_async', 'setpts', 'trim', 'unicode_literals', 'vflip', 'view', 'zoompan']


In [13]:
# 显示报错发生的 Python 文件路径
!cat /content/CodeFormer/basicsr/utils/video_util.py


'''
The code is modified from the Real-ESRGAN:
https://github.com/xinntao/Real-ESRGAN/blob/master/inference_realesrgan_video.py

'''
import cv2
import sys
import numpy as np

try:
    import ffmpeg
except ImportError:
    import pip
    pip.main(['install', '--user', 'ffmpeg-python'])
    import ffmpeg

def get_video_meta_info(video_path):
    ret = {}
    probe = ffmpeg.probe(video_path)
    video_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'video']
    has_audio = any(stream['codec_type'] == 'audio' for stream in probe['streams'])
    ret['width'] = video_streams[0]['width']
    ret['height'] = video_streams[0]['height']
    ret['fps'] = eval(video_streams[0]['avg_frame_rate'])
    ret['audio'] = ffmpeg.input(video_path).audio if has_audio else None
    ret['nb_frames'] = int(video_streams[0]['nb_frames'])
    return ret

class VideoReader:
    def __init__(self, video_path):
        self.paths = []  # for image&folder type
        self.audio = None
  

In [17]:
#@title # Inference the uploaded files
#@markdown `CODEFORMER_FIDELITY`: Balance the quality (lower number) and fidelity (higher number)<br>
# you can add '--bg_upsampler realesrgan' to enhance the background
CODEFORMER_FIDELITY = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown `BACKGROUND_ENHANCE`: Enhance background image with Real-ESRGAN<br>
BACKGROUND_ENHANCE = True #@param {type:"boolean"}
#@markdown `FACE_UPSAMPLE`: Upsample restored faces for high-resolution AI-created images<br>
FACE_UPSAMPLE = False #@param {type:"boolean"}
#@markdown `HAS_ALIGNED`: Input are cropped and aligned faces<br>
HAS_ALIGNED = False #@param {type:"boolean"}
#@markdown `UPSCALE`: The final upsampling scale of the image. Default: 2<br>
UPSCALE = 2 #@param {type:"slider", min:2, max:8, step:1}
#@markdown `DETECTION_MODEL`: Face detector. Default: retinaface_resnet50<br>
DETECTION_MODEL = "retinaface_resnet50" #@param ["retinaface_resnet50", "retinaface_mobile0.25", "YOLOv5l", "YOLOv5n", "dlib"]
#@markdown `DRAW_BOX`: Draw the bounding box for the detected faces.
DRAW_BOX = False #@param {type:"boolean"}

# 参数转换
BACKGROUND_ENHANCE = '--bg_upsampler realesrgan' if BACKGROUND_ENHANCE else ''
FACE_UPSAMPLE = '--face_upsample' if FACE_UPSAMPLE else ''
HAS_ALIGNED = '--has_aligned' if HAS_ALIGNED else ''
DRAW_BOX = '--draw_box' if DRAW_BOX else ''

# ⛏️ 打印调试路径
print("🔍 当前文件路径:", filename)

# ✅ 自动识别类型并使用正确路径
if filename.endswith('.mp4'):
    # 视频路径直接使用完整路径
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path "{filename}" {BACKGROUND_ENHANCE} {FACE_UPSAMPLE} {HAS_ALIGNED} --upscale {UPSCALE} --detection_model {DETECTION_MODEL} {DRAW_BOX}
else:
    # 图片文件放在文件夹中
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path "inputs/user_upload" {BACKGROUND_ENHANCE} {FACE_UPSAMPLE} {HAS_ALIGNED} --upscale {UPSCALE} --detection_model {DETECTION_MODEL} {DRAW_BOX}


🔍 当前文件路径: /content/CodeFormer/inputs/user_upload/video.mp4
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:00<00:00, 122MB/s]
/content/CodeFormer/inference_codeformer.py:49: RuntimeWarning: Running on CPU now! Make sure your PyTorch version matches your CUDA.The unoptimized RealESRGAN is slow on CPU. If you want to disable it, please remove `--bg_upsampler` and `--face_upsample` in command.
  warnings.warn('Running on CPU now! Make sure your PyTorch version matches your CUDA.'
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/60] Processing: video_000000
	detect 1 faces
[2/60] Processing: video_000001
	detect 1 faces
[3/60] Processing: video_000002
	detect 1 faces
[4/60] Processing: video_000003
	detect 1 faces
[5/60] Processing: video_000004
	detect 1 faces
[6/60] Processing: video_000005
	detect 1 faces

In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [19]:
import shutil
import os

# 设置源路径和目标路径
source_path = '/content/CodeFormer/results/video_0.7/final_results'
backup_path = '/content/drive/My Drive/CodeFormer_Backup'

# 创建备份目录（如果不存在）
os.makedirs(backup_path, exist_ok=True)

# 复制所有文件到 Google Drive
for file_name in os.listdir(source_path):
    full_file_name = os.path.join(source_path, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, backup_path)

print("✅ 文件已成功备份到 Google Drive")


✅ 文件已成功备份到 Google Drive


In [15]:
#@title Visualize the results only if are images

import os
import glob

input_folder = 'inputs/user_upload'
result_folder = f'results/user_upload_{CODEFORMER_FIDELITY}/final_results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  basename = os.path.splitext(os.path.basename(input_path))[0]
  output_path = os.path.join(result_folder, basename+'.png')
  img_output = imread(output_path)
  display(img_input, img_output)

In [20]:
#@title Download results
!ls results
print('Download results')
os.system(f'zip -r results.zip results/user_upload_{CODEFORMER_FIDELITY}/final_results')
try:
  files.download("results.zip")
except:
  files.download(f'/content/CodeFormer/results/{filename[:-4]}_{CODEFORMER_FIDELITY}/{filename}')

video_0.7
Download results


FileNotFoundError: Cannot find file: /content/CodeFormer/results//content/CodeFormer/inputs/user_upload/video_0.7//content/CodeFormer/inputs/user_upload/video.mp4